In [1]:
using StatisticalRethinking, Distributed, JLD
using Mamba

# Data
line = Dict{Symbol, Any}()

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [2]:
df2 = filter(row -> row[:age] >= 18, df);
line[:x] = convert(Array{Float64,1}, df2[:weight]);
line[:y] = convert(Array{Float64,1}, df2[:height]);
line[:xmat] = convert(Array{Float64,2}, [ones(length(line[:x])) line[:x]])

# Model Specification
model = Model(
  y = Stochastic(1,
    (xmat, beta, s2) -> MvNormal(xmat * beta, sqrt(s2)),
    false
  ),
  beta = Stochastic(1, () -> MvNormal([178, 0], [sqrt(10000), sqrt(100)])),
  s2 = Stochastic(() -> Uniform(0, 50))
)

# Initial Values
inits = [
  Dict{Symbol, Any}(
    :y => line[:y],
    :beta => [rand(Normal(178, 100)), rand(Normal(0, 10))],
    :s2 => rand(Uniform(0, 50))
  )
  for i in 1:3
]

# Tuning Parameters
scale1 = [0.5, 0.25]
summary1 = identity
eps1 = 0.5

scale2 = 0.5
summary2 = x -> [mean(x); sqrt(var(x))]
eps2 = 0.1

# Define sampling scheme

scheme = [
  Mamba.NUTS([:beta]),
  Mamba.Slice([:s2], 10)
]

setsamplers!(model, scheme)

# MCMC Simulation

sim = mcmc(model, line, inits, 10000, burnin=1000, chains=3)

MCMC Simulation of 10000 Iterations x 3 Chains...

Chain 1:   0% [0:14:17 of 0:14:18 remaining]
Chain 1:  10% [0:01:11 of 0:01:19 remaining]
Chain 1:  20% [0:01:06 of 0:01:23 remaining]
Chain 1:  30% [0:00:57 of 0:01:22 remaining]
Chain 1:  40% [0:00:49 of 0:01:22 remaining]
Chain 1:  50% [0:00:41 of 0:01:22 remaining]
Chain 1:  60% [0:00:33 of 0:01:21 remaining]
Chain 1:  70% [0:00:24 of 0:01:21 remaining]
Chain 1:  80% [0:00:16 of 0:01:21 remaining]
Chain 1:  90% [0:00:08 of 0:01:21 remaining]
Chain 1: 100% [0:00:00 of 0:01:21 remaining]

Chain 2:   0% [0:00:02 of 0:00:02 remaining]
Chain 2:  10% [0:00:49 of 0:00:55 remaining]
Chain 2:  20% [0:00:50 of 0:01:02 remaining]
Chain 2:  30% [0:00:47 of 0:01:08 remaining]
Chain 2:  40% [0:00:41 of 0:01:09 remaining]
Chain 2:  50% [0:00:34 of 0:01:08 remaining]
Chain 2:  60% [0:00:28 of 0:01:09 remaining]
Chain 2:  70% [0:00:21 of 0:01:10 remaining]
Chain 2:  80% [0:00:14 of 0:01:10 remaining]
Chain 2:  90% [0:00:07 of 0:01:11 remaining]
Cha

Object of type "Mamba.ModelChains"

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

[31.7773 114.764 0.88724; 30.7371 115.723 0.860626; … ; 25.4733 113.216 0.913256; 24.1622 113.214 0.914923]

[27.7282 114.518 0.894792; 24.3605 114.705 0.897272; … ; 24.5453 113.955 0.906288; 25.1436 114.96 0.88401]

[25.9549 115.275 0.861863; 27.1983 115.276 0.871713; … ; 24.6864 115.761 0.876158; 29.129 115.773 0.870622]

Show draws summary

In [3]:
describe(sim)

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

Empirical Posterior Estimates:
            Mean         SD       Naive SE        MCSE        ESS   
     s2  26.1828720 1.997823782 0.0121583683 0.02084308379 9000.0000
beta[1] 113.9248155 1.892977987 0.0115202972 0.03568411866 2814.1107
beta[2]   0.9040897 0.041607665 0.0002532162 0.00077483595 2883.5478

Quantiles:
            2.5%        25.0%        50.0%       75.0%        97.5%   
     s2  22.5727091  24.78118942  26.08121669  27.4799122  30.34983118
beta[1] 110.2481198 112.62558110 113.93681061 115.2106722 117.56593597
beta[2]   0.8239225   0.87539109   0.90385915   0.9326315   0.98448512



End of `clip_38.0m.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*